In [ ]:
import sys

sys.path.append("..")

from core.DataLoader import DataPreprocessor, get_load_config_from_yaml
import numpy as np
from importlib import reload
import atlasplots as plt
import ROOT as root
import yaml
import core.keras_models as Models
import core
import os


if False:
    PLOTS_DIR = "plots/ToponiumInfusedML/"
    MODEL_DIR = "models/mixed_model"
elif True:
    PLOTS_DIR = "plots/Testing/"
else:
    PLOTS_DIR = "plots/RegressionML_/"
    MODEL_DIR = "../models/regression_transformer_old/"


CONFIG_PATH = "../config/workspace_config.yaml"

# Create directories
os.makedirs(PLOTS_DIR, exist_ok=True)


from core.utils import lorentz_vector_from_PtEtaPhiE_array
from core.evaluation import c_hel, c_han

# Load data
load_config = get_load_config_from_yaml(CONFIG_PATH)
DataProcessor = DataPreprocessor(load_config)
data_config = DataProcessor.load_from_npz(
    load_config.data_path["test"], event_numbers="odd", max_events=2_000_000
)
DataProcessor.add_custom_feature(
    (
        lambda X, data_config: c_hel(
            lorentz_vector_from_PtEtaPhiE_array(X["top_truth"][:, 0, :4]),
            lorentz_vector_from_PtEtaPhiE_array(X["top_truth"][:, 1, :4]),
            lorentz_vector_from_PtEtaPhiE_array(X["lepton_truth"][:, 0, :4]),
            lorentz_vector_from_PtEtaPhiE_array(X["lepton_truth"][:, 1, :4]),
        )
    ),
    "c_hel",
)
DataProcessor.add_custom_feature(
    (
        lambda X, data_config: c_han(
            lorentz_vector_from_PtEtaPhiE_array(X["top_truth"][:, 0, :4]),
            lorentz_vector_from_PtEtaPhiE_array(X["top_truth"][:, 1, :4]),
            lorentz_vector_from_PtEtaPhiE_array(X["lepton_truth"][:, 0, :4]),
            lorentz_vector_from_PtEtaPhiE_array(X["lepton_truth"][:, 1, :4]),
        )
    ),
    "c_han",
)

X, y = DataProcessor.get_data()
del DataProcessor

In [ ]:
import core.reconstruction.BaselineMethods as BaselineMethods
import core.evaluation.reconstruction_evaluator as Evaluation
from core.reconstruction import (
    GroundTruthReconstructor,
    PerfectAssignmentReconstructor,
    CompositeNeutrinoComponentReconstructor,
)

reload(Evaluation)
reload(BaselineMethods)
nu_flows = GroundTruthReconstructor(data_config, use_nu_flows=True)
true_nu = GroundTruthReconstructor(data_config, use_nu_flows=False)
transformer = PerfectAssignmentReconstructor(
    data_config, neutrino_reco_name=r"ML (MagDir) $\nu$"
)
MODEL_DIR = "../models/regression_transformer/"
transformer.load_model(f"{MODEL_DIR}/odd_model.keras")
transformer_cartesian = PerfectAssignmentReconstructor(
    data_config, neutrino_reco_name=r"ML (Cartesian) $\nu$"
)
MODEL_DIR = "../models/regression_transformer_cartesian/"
transformer_cartesian.load_model(f"{MODEL_DIR}/odd_model.keras")

transformer_PtEtaPhi = PerfectAssignmentReconstructor(
    data_config, neutrino_reco_name=r"ML (PtEtaPhi) $\nu$"
)
MODEL_DIR = "../models/regression_transformer_PtEtaPhi/"
transformer_PtEtaPhi.load_model(f"{MODEL_DIR}/odd_model.keras")


prediction_manager = Evaluation.PredictionManager(
    [true_nu, nu_flows, transformer, transformer_cartesian, transformer_PtEtaPhi],
    X,
    y,  # , load_directory=PREDICTIONS_DIR
)

reco_variable_handler = Evaluation.ReconstructionVariableHandler(
    {}, prediction_manager, X
)

In [ ]:
PLOTS_DIR = "plots/Testing/"
os.makedirs(PLOTS_DIR, exist_ok=True)

In [ ]:
lepton, jet, neutrino = reco_variable_handler.get_reconstructed_4vectors(0)
lepton, jet, neutrino_nu_flows = reco_variable_handler.get_reconstructed_4vectors(1)
lepton, jet, neutrino_nu_transformer = reco_variable_handler.get_reconstructed_4vectors(
    2
)
lepton, jet, neutrino_transformer_cartesian = (
    reco_variable_handler.get_reconstructed_4vectors(3)
)
lepton, jet, neutrino_transformer_PtEtaPhi = (
    reco_variable_handler.get_reconstructed_4vectors(4)
)

In [ ]:
top_vectors = lepton + jet + neutrino
top_vectors_nu_flows = lepton + jet + neutrino_nu_flows
top_vectors_nu_transformer = lepton + jet + neutrino_nu_transformer
top_vectors_transformer_cartesian = lepton + jet + neutrino_transformer_cartesian
top_vectors_transformer_PtEtaPhi = lepton + jet + neutrino_transformer_PtEtaPhi
W_vectors = lepton + neutrino
W_vectors_nu_flows = lepton + neutrino_nu_flows
W_vectors_nu_transformer = lepton + neutrino_nu_transformer
W_vectors_transformer_cartesian = lepton + neutrino_transformer_cartesian
W_vectors_transformer_PtEtaPhi = lepton + neutrino_transformer_PtEtaPhi

In [ ]:
def plot_histogram_comparison(variable_list, labels, xlabel, filename, xlim=None,bins=50):
    root.gStyle.SetPalette(root.kBird)
    palette = root.TColor.GetPalette()
    ncolors = palette.GetSize()
    

    if xlim is None:
        xlim = (min([var.min() for var in variable_list]), max([var.max() for var in variable_list]))
    fig, ax = plt.subplots()
    hists = []
    for idx, (variable, label) in enumerate(zip(variable_list, labels)):
        hist, edges = np.histogram(variable, bins=bins, range=xlim, density=True)
        centers = (edges[:-1] + edges[1:]) / 2
        TH1 = root.TH1F(label, label, bins, xlim[0], xlim[1])
        for i in range(len(hist)):
            TH1.SetBinContent(i + 1, hist[i])  # ROOT bins are 1-indexed
        hists.append(TH1)

    ax.add_margins(top=0.15)

    plt.atlas_label(text="Simulation", loc="upper left")
    max_name_length = max(len(label) for label in labels)
    num_names = len(labels)
    for idx, hist in enumerate(hists):
        ax.plot(hist, label=hist.GetName(), linecolor=palette.At(int(idx*ncolors/num_names)), labelfmt="F")
    ax.set_xlabel(xlabel)
    ax.set_ylabel("arbitrary units", titleoffset=2)
    ax.legend(loc=(0.6, 0.92-num_names*0.05, 0.95, 0.92))

    fig.savefig(f"{PLOTS_DIR}/{filename}")


names = [
    r"True",
    r"NuFlows",
    r"ML (MagDir)",
    r"ML (Cartesian)",
    r"ML (PtEtaPhi)",
]

names_dev = [
    r"NuFlows",
    r"ML (MagDir)",
    r"ML (Cartesian)",
    r"ML (PtEtaPhi)",
]

In [ ]:
ttbar = top_vectors[:, 0, :] + top_vectors[:, 1, :]
ttbar_nu_flows = top_vectors_nu_flows[:, 0, :] + top_vectors_nu_flows[:, 1, :]
ttbar_nu_transformer = (
    top_vectors_nu_transformer[:, 0, :] + top_vectors_nu_transformer[:, 1, :]
)
ttbar_transformer_cartesian = (
    top_vectors_transformer_cartesian[:, 0, :]
    + top_vectors_transformer_cartesian[:, 1, :]
)
ttbar_transformer_PtEtaPhi = (
    top_vectors_transformer_PtEtaPhi[:, 0, :]
    + top_vectors_transformer_PtEtaPhi[:, 1, :]
)
from core.utils import compute_mass_from_lorentz_vector_array

ttbar_mass = compute_mass_from_lorentz_vector_array(ttbar) / 1e3
ttbar_mass_nu_flows = compute_mass_from_lorentz_vector_array(ttbar_nu_flows) / 1e3
ttbar_mass_nu_transformer = (
    compute_mass_from_lorentz_vector_array(ttbar_nu_transformer) / 1e3
)
ttbar_mass_transformer_cartesian = (
    compute_mass_from_lorentz_vector_array(ttbar_transformer_cartesian) / 1e3
)
ttbar_mass_transformer_PtEtaPhi = (
    compute_mass_from_lorentz_vector_array(ttbar_transformer_PtEtaPhi) / 1e3
)
plot_histogram_comparison(
    [
        ttbar_mass,
        ttbar_mass_nu_flows,
        ttbar_mass_nu_transformer,
        ttbar_mass_transformer_cartesian,
        ttbar_mass_transformer_PtEtaPhi,
    ],
    names,
    r"m(t#bar{t}) [GeV]",
    "ttbar_mass_comparison.pdf",
    xlim=(200, 1200),
)

In [ ]:
ttbar_mass_dev_nu_flows = ttbar_mass_nu_flows - ttbar_mass
ttbar_mass_dev_nu_transformer = ttbar_mass_nu_transformer - ttbar_mass
ttbar_mass_dev_transformer_cartesian = ttbar_mass_transformer_cartesian - ttbar_mass
ttbar_mass_dev_transformer_PtEtaPhi = ttbar_mass_transformer_PtEtaPhi - ttbar_mass

In [ ]:
W_mass = compute_mass_from_lorentz_vector_array(W_vectors)[..., 0] / 1e3
W_mass_nu_flows = (
    compute_mass_from_lorentz_vector_array(W_vectors_nu_flows)[..., 0] / 1e3
)
W_mass_nu_transformer = (
    compute_mass_from_lorentz_vector_array(W_vectors_nu_transformer)[..., 0] / 1e3
)
W_mass_transformer_cartesian = (
    compute_mass_from_lorentz_vector_array(W_vectors_transformer_cartesian)[..., 0]
    / 1e3
)
W_mass_transformer_PtEtaPhi = (
    compute_mass_from_lorentz_vector_array(W_vectors_transformer_PtEtaPhi)[..., 0] / 1e3
)
plot_histogram_comparison(
    [
        W_mass,
        W_mass_nu_flows,
        W_mass_nu_transformer,
        W_mass_transformer_cartesian,
        W_mass_transformer_PtEtaPhi,
    ],
    names,
    r"m(W)[GeV]",
    "W_mass_comparison.pdf",
    xlim=(0, 200),
)

In [ ]:
dev_nu_px_nu_flows = (neutrino_nu_flows[..., 0] - neutrino[..., 0])[..., 0] / 1e3
dev_nu_px_nu_transformer = (neutrino_nu_transformer[..., 0] - neutrino[..., 0])[
    ..., 0
] / 1e3
dev_nu_px_transformer_cartesian = (
    neutrino_transformer_cartesian[..., 0] - neutrino[..., 0]
)[..., 0] / 1e3
dev_nu_px_transformer_PtEtaPhi = (
    neutrino_transformer_PtEtaPhi[..., 0] - neutrino[..., 0]
)[..., 0] / 1e3

dev_nu_py_nu_flows = (neutrino_nu_flows[..., 1] - neutrino[..., 1])[..., 0] / 1e3
dev_nu_py_nu_transformer = (neutrino_nu_transformer[..., 1] - neutrino[..., 1])[
    ..., 0
] / 1e3
dev_nu_py_transformer_cartesian = (
    neutrino_transformer_cartesian[..., 1] - neutrino[..., 1]
)[..., 0] / 1e3
dev_nu_py_transformer_PtEtaPhi = (
    neutrino_transformer_PtEtaPhi[..., 1] - neutrino[..., 1]
)[..., 0] / 1e3

dev_nu_pz_nu_flows = (neutrino_nu_flows[..., 2] - neutrino[..., 2])[..., 0] / 1e3
dev_nu_pz_nu_transformer = (neutrino_nu_transformer[..., 2] - neutrino[..., 2])[
    ..., 0
] / 1e3
dev_nu_pz_transformer_cartesian = (
    neutrino_transformer_cartesian[..., 2] - neutrino[..., 2]
)[..., 0] / 1e3
dev_nu_pz_transformer_PtEtaPhi = (
    neutrino_transformer_PtEtaPhi[..., 2] - neutrino[..., 2]
)[..., 0] / 1e3
plot_histogram_comparison(
    [
        dev_nu_px_nu_flows,
        dev_nu_px_nu_transformer,
        dev_nu_px_transformer_cartesian,
        dev_nu_px_transformer_PtEtaPhi,
    ],
    names_dev,
    r"\Delta p_{x}(\nu) [GeV]",
    "nu_px_deviation_comparison.pdf",
    xlim=(-100, 100),
)
plot_histogram_comparison(
    [
        dev_nu_py_nu_flows,
        dev_nu_py_nu_transformer,
        dev_nu_py_transformer_cartesian,
        dev_nu_py_transformer_PtEtaPhi,
    ],
    names_dev,
    r"\Delta p_{y}(\nu) [GeV]",
    "nu_py_deviation_comparison.pdf",
    xlim=(-100, 100),
)
plot_histogram_comparison(
    [
        dev_nu_pz_nu_flows,
        dev_nu_pz_nu_transformer,
        dev_nu_pz_transformer_cartesian,
        dev_nu_pz_transformer_PtEtaPhi,
    ],
    names_dev,
    r"\Delta p_{z}(\nu) [GeV]",
    "nu_pz_deviation_comparison.pdf",
    xlim=(-300, 300),
)

plot_histogram_comparison(
    [
        np.abs(dev_nu_px_nu_flows),
        np.abs(dev_nu_px_nu_transformer),
        np.abs(dev_nu_px_transformer_cartesian),
        np.abs(dev_nu_px_transformer_PtEtaPhi),
    ],
    names_dev,
    r"|\Delta p_{x}(\nu)| [GeV]",
    "nu_px_absolute_deviation_comparison.pdf",
    xlim=(0, 100),
)
plot_histogram_comparison(
    [
        np.abs(dev_nu_py_nu_flows),
        np.abs(dev_nu_py_nu_transformer),
        np.abs(dev_nu_py_transformer_cartesian),
        np.abs(dev_nu_py_transformer_PtEtaPhi),
    ],
    names_dev,
    r"|\Delta p_{y}(\nu)| [GeV]",
    "nu_py_absolute_deviation_comparison.pdf",
    xlim=(0, 100),
)
plot_histogram_comparison(
    [
        np.abs(dev_nu_pz_nu_flows),
        np.abs(dev_nu_pz_nu_transformer),
        np.abs(dev_nu_pz_transformer_cartesian),
        np.abs(dev_nu_pz_transformer_PtEtaPhi),
    ],
    names_dev,
    r"|\Delta p_{z}(\nu)| [GeV]",
    "nu_pz_absolute_deviation_comparison.pdf",
    xlim=(0, 300),
)


In [ ]:
W_mass_dev_nu_flows = W_mass_nu_flows - W_mass
W_mass_dev_nu_transformer = W_mass_nu_transformer - W_mass
W_mass_dev_transformer_cartesian = W_mass_transformer_cartesian - W_mass
W_mass_dev_transformer_PtEtaPhi = W_mass_transformer_PtEtaPhi - W_mass

In [ ]:
nu_mag = np.linalg.norm(neutrino, axis=-1)[..., 0] / 1e3
nu_flows_mag = np.linalg.norm(neutrino_nu_flows, axis=-1)[..., 0] / 1e3
nu_transformer_mag = np.linalg.norm(neutrino_nu_transformer, axis=-1)[..., 0] / 1e3
nu_transformer_cartesian_mag = (
    np.linalg.norm(neutrino_transformer_cartesian, axis=-1)[..., 0] / 1e3
)
nu_transformer_PtEtaPhi_mag = (
    np.linalg.norm(neutrino_transformer_PtEtaPhi, axis=-1)[..., 0] / 1e3
)
nu_mag_dev_nu_flows = nu_flows_mag - nu_mag
nu_mag_dev_nu_transformer = nu_transformer_mag - nu_mag
nu_mag_dev_transformer_cartesian = nu_transformer_cartesian_mag - nu_mag
nu_mag_dev_transformer_PtEtaPhi = nu_transformer_PtEtaPhi_mag - nu_mag

plot_histogram_comparison(
    [
        nu_mag_dev_nu_flows,
        nu_mag_dev_nu_transformer,
        nu_mag_dev_transformer_cartesian,
        nu_mag_dev_transformer_PtEtaPhi,
    ],
    names_dev,
    r"\Delta|p(\nu)| [GeV]",
    "nu_magnitude_deviation_comparison.pdf",
    xlim=(-100, 100),
)
plot_histogram_comparison(
    [
        nu_mag,
        nu_flows_mag,
        nu_transformer_mag,
        nu_transformer_cartesian_mag,
        nu_transformer_PtEtaPhi_mag,
    ],
    names,
    r"|p(\nu)| [GeV]",
    "nu_magnitude_comparison.pdf",
    xlim=(0, 500),
)

In [ ]:
def cos_lep_nu(lepton, neutrino):
    return np.sum(lepton[..., :3] * neutrino[..., :3], axis=-1) / (
        np.linalg.norm(lepton[..., :3], axis=-1)
        * np.linalg.norm(neutrino[..., :3], axis=-1)
    )


cos_lep_nu_true = cos_lep_nu(lepton, neutrino)[..., 0]
cos_lep_nu_nu_flows = cos_lep_nu(lepton, neutrino_nu_flows)[..., 0]
cos_lep_nu_nu_transformer = cos_lep_nu(lepton, neutrino_nu_transformer)[..., 0]
cos_lep_nu_transformer_cartesian = cos_lep_nu(lepton, neutrino_transformer_cartesian)[
    ..., 0
]
cos_lep_nu_transformer_PtEtaPhi = cos_lep_nu(lepton, neutrino_transformer_PtEtaPhi)[
    ..., 0
]
plot_histogram_comparison(
    [
        cos_lep_nu_true,
        cos_lep_nu_nu_flows,
        cos_lep_nu_nu_transformer,
        cos_lep_nu_transformer_cartesian,
        cos_lep_nu_transformer_PtEtaPhi,
    ],
    names,
    r"cos(#theta_{#nu,lep})",
    "cos_lep_nu_comparison.pdf",
    xlim=(-1, 1),
)

In [ ]:
cos_lep_nu_dev_nu_flows = cos_lep_nu_nu_flows - cos_lep_nu_true
cos_lep_nu_dev_nu_transformer = cos_lep_nu_nu_transformer - cos_lep_nu_true
cos_lep_nu_dev_transformer_cartesian = (
    cos_lep_nu_transformer_cartesian - cos_lep_nu_true
)
cos_lep_nu_dev_transformer_PtEtaPhi = cos_lep_nu_transformer_PtEtaPhi - cos_lep_nu_true

In [ ]:
plot_histogram_comparison(
    [
        cos_lep_nu_dev_nu_flows,
        cos_lep_nu_dev_nu_transformer,
        cos_lep_nu_dev_transformer_cartesian,
        cos_lep_nu_dev_transformer_PtEtaPhi,
    ],
    names_dev,
    r"\Delta cos(\theta_{lep,\nu})",
    "cos_lep_nu_deviation_comparison.pdf",
    xlim=(-2, 2),
)

In [ ]:
cos_nu_nubar_true = cos_lep_nu(neutrino[:, 0, :], neutrino[:, 1, :])
cos_nu_nubar_nu_flows = cos_lep_nu(neutrino_nu_flows[:, 0, :], neutrino_nu_flows[:, 1, :])
cos_nu_nubar_nu_transformer = cos_lep_nu(
    neutrino_nu_transformer[:, 0, :], neutrino_nu_transformer[:, 1, :]
)
cos_nu_nubar_transformer_cartesian = cos_lep_nu(
    neutrino_transformer_cartesian[:, 0, :], neutrino_transformer_cartesian[:, 1, :]
)
cos_nu_nubar_transformer_PtEtaPhi = cos_lep_nu(
    neutrino_transformer_PtEtaPhi[:, 0, :], neutrino_transformer_PtEtaPhi[:, 1, :]
)
plot_histogram_comparison(
    [
        cos_nu_nubar_true,
        cos_nu_nubar_nu_flows,
        cos_nu_nubar_nu_transformer,
        cos_nu_nubar_transformer_cartesian,
        cos_nu_nubar_transformer_PtEtaPhi,
    ],
    names,
    r"cos(\theta_{\nu,\bar{\nu}})",
    "cos_nu_nubar_comparison.pdf",
    xlim=(-1, 1),
)

cos_nu_nubar_dev_nu_flows = cos_nu_nubar_nu_flows - cos_nu_nubar_true
cos_nu_nubar_dev_nu_transformer = cos_nu_nubar_nu_transformer - cos_nu_nubar_true
cos_nu_nubar_dev_transformer_cartesian = (
    cos_nu_nubar_transformer_cartesian - cos_nu_nubar_true
)
cos_nu_nubar_dev_transformer_PtEtaPhi = (
    cos_nu_nubar_transformer_PtEtaPhi - cos_nu_nubar_true
)

plot_histogram_comparison(
    [
        cos_nu_nubar_dev_nu_flows,
        cos_nu_nubar_dev_nu_transformer,
        cos_nu_nubar_dev_transformer_cartesian,
        cos_nu_nubar_dev_transformer_PtEtaPhi,
    ],
    names_dev,
    r"\Delta cos(\theta_{\nu,\bar{\nu}})",
    "cos_nu_nubar_deviation_comparison.pdf",
    xlim=(-1, 1),
)


In [ ]:
nu_system_4vector = neutrino[:, 0, :] + neutrino[:, 1, :]
nu_system_nu_flows_4vector = neutrino_nu_flows[:, 0, :] + neutrino_nu_flows[:, 1, :]
nu_system_nu_transformer_4vector = (
    neutrino_nu_transformer[:, 0, :] + neutrino_nu_transformer[:, 1, :]
)
nu_system_transformer_cartesian_4vector = (
    neutrino_transformer_cartesian[:, 0, :] + neutrino_transformer_cartesian[:, 1, :]
)
nu_system_transformer_PtEtaPhi_4vector = (
    neutrino_transformer_PtEtaPhi[:, 0, :] + neutrino_transformer_PtEtaPhi[:, 1, :]
)
nu_system_mag = np.linalg.norm(nu_system_4vector[..., :3], axis=-1) / 1e3
nu_system_nu_flows_mag = (
    np.linalg.norm(nu_system_nu_flows_4vector[..., :3], axis=-1)/ 1e3
)
nu_system_nu_transformer_mag = (
    np.linalg.norm(nu_system_nu_transformer_4vector[..., :3], axis=-1)/ 1e3
)
nu_system_transformer_cartesian_mag = (
    np.linalg.norm(nu_system_transformer_cartesian_4vector[..., :3], axis=-1) / 1e3
)
nu_system_transformer_PtEtaPhi_mag = (
    np.linalg.norm(nu_system_transformer_PtEtaPhi_4vector[..., :3], axis=-1)/ 1e3
)
nu_system_mag_dev_nu_flows = nu_system_nu_flows_mag - nu_system_mag
nu_system_mag_dev_nu_transformer = nu_system_nu_transformer_mag - nu_system_mag
nu_system_mag_dev_transformer_cartesian = (
    nu_system_transformer_cartesian_mag - nu_system_mag
)
nu_system_mag_dev_transformer_PtEtaPhi = (
    nu_system_transformer_PtEtaPhi_mag - nu_system_mag
)
plot_histogram_comparison(
    [
        nu_system_mag_dev_nu_flows,
        nu_system_mag_dev_nu_transformer,
        nu_system_mag_dev_transformer_cartesian,
        nu_system_mag_dev_transformer_PtEtaPhi,
    ],
    names_dev,
    r"\Delta |p(\nu+\bar{\nu})| [GeV]",
    "nu_system_magnitude_deviation_comparison.pdf",
    xlim=(-100, 100),
)

In [ ]:
bbar_ll_system_4vector = lepton[:, 0, :] + lepton[:, 1, :] + jet[:, 0, :] + jet[:, 1, :]

cos_lep_jet_nu = cos_lep_nu(bbar_ll_system_4vector,nu_system_4vector)
cos_lep_jet_nu_nu_flows = cos_lep_nu(bbar_ll_system_4vector,nu_system_nu_flows_4vector)
cos_lep_jet_nu_nu_transformer = cos_lep_nu(bbar_ll_system_4vector,nu_system_nu_transformer_4vector)
cos_lep_jet_nu_transformer_cartesian = cos_lep_nu(bbar_ll_system_4vector,nu_system_transformer_cartesian_4vector)
cos_lep_jet_nu_transformer_PtEtaPhi = cos_lep_nu(bbar_ll_system_4vector,nu_system_transformer_PtEtaPhi_4vector)
plot_histogram_comparison(
    [
        cos_lep_jet_nu,
        cos_lep_jet_nu_nu_flows,
        cos_lep_jet_nu_nu_transformer,
        cos_lep_jet_nu_transformer_cartesian,
        cos_lep_jet_nu_transformer_PtEtaPhi,
    ],
    names,
    r"cos(\theta(\nu\bar{\nu},b\bar{b}lep^+lep^-))",
    "cos_lep_jet_nu_comparison.pdf",
    xlim=(-1, 1),
)

In [ ]:
cos_lep_jet_dev_nu_flows = cos_lep_jet_nu_nu_flows - cos_lep_jet_nu
cos_lep_jet_dev_nu_transformer = cos_lep_jet_nu_nu_transformer - cos_lep_jet_nu
cos_lep_jet_dev_transformer_cartesian = cos_lep_jet_nu_transformer_cartesian - cos_lep_jet_nu
cos_lep_jet_dev_transformer_PtEtaPhi = cos_lep_jet_nu_transformer_PtEtaPhi - cos_lep_jet_nu
plot_histogram_comparison(
    [
        cos_lep_jet_dev_nu_flows,
        cos_lep_jet_dev_nu_transformer,
        cos_lep_jet_dev_transformer_cartesian,
        cos_lep_jet_dev_transformer_PtEtaPhi,
    ],
    names_dev,
    r"\Delta cos(\theta(\nu\bar{\nu},b\bar{b}lep^+lep^-))",
    "cos_lep_jet_nu_deviation_comparison.pdf",
    xlim=(-1, 1),
)